In [ ]:
import math, copy, os, time, enum, argparse

import matplotlib.pyplot as plt
import seaborn

import torch
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
from torch.hub import download_url_to_file

from torchtext.data import Dataset, BucketIterator, Field, Example
from torchtext.data.utils import interleave_keys
from torchtext import datasets
from torchtext.data import Example
import spacy

from nltk.translate.bleu_score import corpus_bleu

In [ ]:
BASELINE_MODEL_NUMBER_OF_LAYERS = 6
BASELINE_MODEL_DIMENSION = 512
BASELINE_MODEL_NUMBER_OF_HEADS = 8
BASELINE_MODEL_DROPOUT_PROB = 0.1
BASELINE_MODEL_LABEL_SMOOTHING_VALUE = 0.1

CHECKPOINTS_PATH = os.path.join(os.getcwd(), 'models', 'checkpoints')
BINARIES_PATH = os.path.join(os.getcwd(), 'models', 'binaries')
DATA_DIR_PATH = os.path.join(os.getcwd(), 'data')

os.makedirs(CHECKPOINTS_PATH, exist_ok = True)
os.makedirs(BINARIES_PATH, exist_ok = True)
os.makedirs(DATA_DIR_PATH, exist_ok = True)

BOS_TOKEN = '<s>'
EOS_TOKEN = '</s>'
PAD_TOKEN = '<pad'

# PART 1 : Understanding the model.

In [ ]:
class Transformer(torch.nn.Module):
    def __init__(self, model_dim, src_vocab_size, tar_vocab_size, n_heads, n_layers, p_dropout, log_att_w = False):
        super().__init__()

        # Embeds source/target token ids into embedding vectors.
        self.src_embedding = Embedding(src_vocab_size, model_dim)
        self.tar_embedding = Embedding(trg_vocab_size, model_dim)

        # Positional embedding.
        self.src_pos_embedding = PositionalEncoding(model_dim, p_dropout)
        self.tar_pos_embedding = PositionalEncoding(model_dim, p_dropout)

        # Embedding get deep-copied multiple times.
        mha = MultiHeadAttention(model_dim, n_heads, p_dropout)
        pwn = PositionwiseFeedForwardNet(moidel_dim, p_dropout)
        encoder_layer = EncoderLayer(model_dim, p_dropout, mha, pwn)
        decoder_layer = DecoderLayer(model_dim, p_dropout, mha, pwn)

        # Encoder/Decoder stacks.
        self.encoder = Encoder(encoder_layer, n_layers)
        self.decoder = Decoder(decoder_layer, n_layers)

        # To convert target token representations into log probability vectors of tar vocabulary size.
        # We use log probability vectors because torch's KLDivLoss expects log probabilities.
        self.decoder_generator = DecoderGenerator(model_dim, tar_vocab_size)
        self.init_params()

    def init_params(self):
        for name, p in self.model.parameters():
            if p_dim() > 1:
                torch.nn.init.xavier_unifrom(p)

    def forward(self, src_token_ids_batch, tar_token_ids_batch, src_mask, tar_mask):
        src_repr_batch = self.encode(src_token_ids_batch, src_mask)
        tar_log_probs = self.decode(tar_token_ids_batch, src_repr_batch, tar_mask, src_mask)
        return tar_log_probs

    def encode(self, src_token_ids_batch, src_mask):
        """

        Args:
            src_token_ids_batch ([B, S, D]): [batch_size, longest_src_token, model_dim] the encoder stack preserves this shape.
            src_mask ([type]): [description]
        """
        src_embeddings_batch = self.src_embedding(src_token_ids_batch) # get embedding vectors for src token ids.
        src_embeddings_batch = self.src_pos_embedding(src_embeddings_batch) # add positional embedding.
        src_repr_batch = self.encoder(src_embeddings_batch, src_mask) # Forward pass through the encoder.
        return src_repr_batch

    def decode(self, tar_token_ids_batch, src_repr_batch, tar_mask, src_mask):
        tar_embeddings_batch = self.tar_embedding(tar_token_ids_batch, src_repr_batch, tar_mask, src_mask) # get embedding vector for tar token ids.
        tar_embeddings_batch = self.tar_pos_embedding(tar_embeddings_batch) # add positional embedding.

        # shape (b, t, d) batch_size, longest tar token sequence length, model_dim
        tar_repr_batch = self.decoder(tar_embedding_batch, src_repr_batch, tar_mask, src_mask)
        # (b, t, v) withg v the vocab size.
        # decoder generator does linear projection + log softmax.
        tar_log_probs = self.decoder_generator(tar_representations_batch)
        # reshape into (b*t, v) to pass in KL-div loss.
        tar_log_probs = tar_log_probs.reshape(-1, tar_log_probs.shape[-1])
        return tar_log_probs

1. tokens get embedded into source sequences.
2. encoder takes a batch of the source sequences.
3. encoder mixes sources sequences through 6 layers of the base transformer via attention.
4. the final output gets consumed by the decoder.

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self, encoder_layer, n_layers):
        super().__init__()
        assert isinstance(encoder_layer, EncoderLayer), f'Expected Encoder layer, got {type(encoder_layer)} instead !'

        # get a list of of the encoder layers.
        self.encoder_layers = get_clones(encoder_layer, n_layers)
        self.norm = torch.nn.LayerNorm(encoder_layer.model_dim)

    def forward(self, src_embeddings_batch, src_mask):
        src_representations_batch = src_embeddings_batch
        # the role of the source mask is to ignore padded token representations in the multi head self attention module.
        for encoder_layer in self.encoder_layers:
            src_representations_batch = encoder_layer(src_representations_batch, src_mask)
        return self.norm(src_representations_batch)

In [ ]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, model_dim, p_dropout, multihead_att, pointwise_net):
        super().__init__()
        n_sublayers_encoder = 2
        self.sublayers = get_clones(SublayerLogic(model_dim, p_dropout), n_sublayers_encoder)

        self.multihead_att = multihead_att
        self.pointwise_net = pointwise_net

        self.model_dimension = model_dimension
    
    def forward(self, src_repr_batch, src_mask):
        # define a lambda function that takes src_repr_batch as input to have a uniform inteface for the sublayer logic.
        encoder_self_attention = lambda srb: self.multihead_att(query = srb, key = srb, value = srb, mask = src_mask)
        # self-attention mha sublayer followed by pointwise feed forward sublayer.
        # sublayerLogic takes as input the data and the logic it should execute (attention/feedforward)
        src_repr_batch = self.sublayers[0](src_repr_batch, encoder_self_attention)
        src_repr_batch = self.sublayers[1](src_repr_batch, self.pointwise_net)

        return src_repr_batch


 1. target sequences with embedded tokens.
 2. 6 iterations of mixing via attention while attending to source token representations.
 3. final output sends target token representations into the decoder generator.
 4. target tokens are converted to log probabilities.

The decoder uses causal masking to prevent tokens from looking into the future.

<img src="images/causal_mask.PNG">

In [ ]:
class Decoder(torch.nn.Module):
    def __init__(self, decoder_layer, n_layers):
        super().__init__()
        assert isinstance(decoder_layer, DecoderLayer), f'Expected DecoderLayer, got {type(decoder_layer)} !'

        self.encoder_layers = get_clones(decoder_layer, n_layers)
        self.norm = torch.nn.LayerNorm(decoder_layer.model_dimension)
    
    def forward(self, tar_embedding_batch, src_repr_batch, tar_mask, src_mask):
        tar_repr_batch = tar_embedding_batch

        # Forward pass through decoder stack.
        for decoder_layer in self.decoder_layers:
            # target mask masks pad tokens + future tokens.
            tar_repr_batch = decoder_layer(tar_repr_batch, src_repr_batch, tar_mask, src_mask)

        return self.norm(tar_repr_batch)
    
class DecoderLayer(torch.nn.Module):
    def __init__(self, model_dim, p_dropout, multihead_att, pointwise_net):
        super().__init__()
        n_sublayers_decoder = 3
        self.sublayers = get_clones(SublayerLogic(model_dim, p_dropout), n_sublayers_decoder)
        self.tar_multihead_att = copy.deepcopy(multihead_att)
        self.src_multihead_att = copy.depcopy(multihead_att)
        self.pointwise_net = pointwise_net
        self.model_dimension = model_dim
    
    def forward(self, tar_repr_batch, src_repr_batch, tar_mask, src_mask):
        # the inputs that are not passed into lambdas (masks and source representation batches) are cached.
        srb = src_repr_batch
        decoder_tar_self_att = lambda trb: self.tar_multihead_att(query = trb, key = trb, vavlue = trb, mask = tar_mask)
        decoder_src_att = lambda trb: self.src_multihead_att(query = trb, key = srb, value = srb, mask = src_mask)

        # self-attention multihead attention sublayer followed by a source-attending multihead attention and pointwise feed forward net sublayer.
        tar_repr_batch = self.sublayers[0](tar_repr_batch, decoder_tar_self_att)
        tar_repr_batch = self.sublayers[1](tar_repr_batch, decoder_src_att)
        tar_repr_batch = self.sublayers[2](tar_repr_batch, self.pointwise_net)
        return tar_repr_batch

the **decoder generator** :
1. Projects the final decoder token representation. D --> V
2. applies log softmax

In [ ]:
class DecoderGenerator(torch.nn.Module):
    def __init__(self, model_dim, vocab_size):
        super().__init__()
        self.linear = torch.nn.Linear(model_dim, vocab_size)
        # linear layer has shape (B, T, V). B batch_size, T max_target_token_sequences, V_target_vocab_size.
        self.log_softmax = torch.nn.LogSoftmax(dim = -1)

    def forward(self, tar_repr_batch):
        return self.log_softmax(self.linear(tar_repr_batch))

# Positional encoding

this is what the positional encoding traditionally looks like : 
<img src="images/pos_encoding.jpg">

In [ ]:
class SublayerLogic(torch.nn.Module):
    def __init__(self, model_dim, p_dropout):
        super().__init__()
        self.norm = torch.nn.LayerNorm(model_dim)
        self.dropout = torch.nn.Dropout(p = p_dropout)
        # In the original paper, layer norm is doe after the residual connection but experiments proved to be more effective before.
    
    def forward(self, repr_batch, sublayer_module):
        return repr_batch + self.dropout(sublayer_module(self.norm(repr_batch)))


In [ ]:
class PositionwiseFeedForwardNet(torch.nn.Module):
    """position-wise because the feed-forward net is independantly applied to every token's representation.
    i.e same as a nested loop going over the batch size and max token sequence length dimensions then applies the network to the sequence representation.

    Args:
        torch ([type]): [description]
    """
    def __init__(self, model_dim, p_dropout, width_mult = 4):
        super().__init__()
        self.linear1 = torch.nn.Linear(model_dim, width_mult*model_dim)
        self.linear2 = torch.nn.Linear(width_mult*model_dim, model_dim)
    
        # dropout layer not mentionned in the paper but commonly used to avoid overfitting.
        self.dropout = torch.nn.Dropout(p = p_dropout)
        self.relu = torch.nn.ReLU()

        # representations batch : (B, S/T, D) (batch_size, max_token_sequence_length, model_dim)
    
    def forward(self, repr_batch):
        return self.linear2(self.dropout(self.relu(self.linear1(repr_batch))))

In [ ]:
class Embedding(torch.nn.Module):
    def __init__(self, vocab_size, model_dim):
        super().__init__()
        self.embeddings_table = torch.nn.Embedding(vocab_size, model_dim)
        self.model_dim = model_dim
    
    def forward(self, token_ids_batch):
        assert token_ids_batch.ndim == 2, f'Expected : (batch_size, max_token_seq_length), got {token_ids_batch.shape}'
        # token_ids_batch has size (B, S/T)
        # final sequence wille be (B, S/T, D) witjh the model dimensions, so every token id has an associated vector.
        embeddings = self.embeddings_table(token_ids_batch)
        # we multiply the embedding weights by the squre root of the model dimension as stated in the paper.
        return embeddings*math.sqrt(self.model_dim)

In [ ]:
class PositionEncoding(torch.nn.Module):
    def __init__(self, model_dim, p_dropout, expected_max_seq_length = 5000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p = p_dropout)
        # as suggested in the paper, use sine frequencies to form a geometric progression as position encodings.
        position_id = torch.arange(0, expected_max_seq_length).unsqueeze(1)
        frequencies = torch.pow(10000., -torch.arange(0, model_dim, 2, dtype = torch.float)/model_dim)
        
        positional_encodings_table = torch.zeros(expected_max_seq_length, model_dim)
        positional_encodings_table[0, 0::2] = torch.sin(position_id*frequencies) # sine on even positions.
        positional_encodings_table[:, 1::2] = torch.cos(position_id*frequencies) # cosine on odd positions.

        # register buffer in order to save the positional encodings table inside the state dict even though these are not trainable.
        # So if we don't register them to the buffer, they will not be saved in the state dict.
        self.register_buffer('positional_encodings_table', positional_encodings_table)

    def forward(self, embeddings_batch):
        assert embeddings_batch.ndim == 3 and embeddings_batch.shape[-1] == self.positional_encodings_table.shape[1], f'Expected (batch_size, max_token_sequence_length, model_dimension and got {embeddings_batch.shape}'
        # embeddings_batch.shape (B, S/T, D)
        # transformed into (S/T, D) that ill be broadcasted to (B, S/T, D) before adding it to the embedding.
        positional_encodings = self.encodings_table[:embeddings_batch.shape[1]]
        # then apply dropout to the sum of the positional encodings and token embeddings.
        return self.dropout(embeddings_batch + positional_encodings)

In [ ]:
def get_clones(module, n_deep_clones):
    # creates independent modules so that each clones weights can be independantly updated.
    return torch.nn.ModuleList([copy.deepcopy(module) for _ in range(n_deep_copies)])

## Multihead Attention

In [ ]:
class MultiHeadAttention(torch.nn.Modue):
    def __init__(self, model_dimension, n_heads, p_dropout, log_att_weights):
        super().__init__()
        assert model_dim%n_heads == 0, 'model dimension must be divisible by the number of heads !'
        self.head_dim = int(model_dim/n_heads)
        self.n_heads = n_heads

        self.qkv_nets = get_clones(torch.nn.Linear(model_dim/n_heads))
        self.out_projection_net = torch.nn.Linear(model_dim, model_dim)
        
        self.attention_dropout = torch.nn.Dropout(p= p_dropout)
        self.softmax = torch.nn.Softmax(dim = -1)

        self.log_att_weights = log_att_weights
        self.attention_weights = None
    
    def attention(self, query, key, value, mask):
        # Step 1 : scaled dot product attentin.
        scores = torch.matmul(query, key.transpose(-2, -1))/math.sqrt(self.head_dim)
        # Step 2 : Optionally mask tokens whose representation we want to ignore by setting a big negative number on loations we want to ignore.
        # mask shape (B, 1 1, S) or (B, A, T, T) will get broadcasted to match the score shape.
        if mask is not None:
            scores.masked_fill_(mask == torch.tensor(False), float('-inf'))
        # Step 3 : Calculate attention weights.
        att_weights = self.softmax(scores)
        # Step 4 : apply dropout to attention weights aswell.
        att_weights = self.attention_dropout(att_weights)
        # Step 5 : calculate new token representations based on attention weights.
        # (B NH, S, HD) --> (B, NH, T, HD)
        intermediate_token_repr = torch.matmul(att_weights, value)
        return intermediate_token_repr, att_weights
    
    def forward(self, query, key, value, mask):
        batch_size = query.shape[0]
        # Step 1 : input linear projection
        query, key, value = [net(x).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2) for net, x in zip(self.qkv_nets, (query, key, value))]
        # Step 2 : apply attention, compare query with key and use it to combine values.
        intermediate_token_repr, att_weights = self.attention(query, key, value, mask)
        # log the attention weights for visualization purposes, can be turned off during training to avoid memory problems.
        # Step 3 : reshape from (B, NH, S/T, HD) over (B, S/T, NH, HD) into (B, S/T, NH*HD) (shape of the input to the forward function).
        reshaped = intermediate_token_repr.transpose(1, 2).reshape(batch_size, -1, self.n_heads*self.head_dim)
        # Step 4 : output linear projection.
        token_repr  = self.out_projection_net(reshaped)
        return token_repr




# Part 2 : Data pipeline

In [ ]:
class DatasetType(enum.Enum):
    IWSLT = 0,
    WMT14 = 1

class LanguageDirection(enum.Enum):
    E2G = 0,
    G2E = 1

In [ ]:
def get_datasets_and_vocabs(dataset_path, language_direction, use_iwslt = True, use_caching_mechanism = True):
    german_to_english = language_direction == LanguageDirection.G2E.name
    spacy_de = spacy.load('de_core_news_sm')
    spacy_en = spacy.load('en_core_web_sm')

    def tokenize_de(text):
        return [tok.text for tok in spacy_de.tokenizer(text)]

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]
    
    src_tokenizer = tokenize_de if german_to_english else tokenize_en
    tar_tokenizer = tokenize_en if german_to_english else tokenize_de
    # batch first then channels.
    src_field_processor = Field(tokenize = src_tokenizer, pad_token = PAD_TOKEN, batch_first = True)
    tar_field_processor = Field(tokenize = tar_tokenizer, init_token = BOS_TOKEN, eos_token = EOS_TOKEN, pad_token = PAD_TOKEN, batch_first = True)

    fields = [('src', src_field_processor), ('tar', tar_field_processor)]
    MAX_LEN = 100 # filter out sequences with more than MAX_LEN tokens.

    filter_pred = lambda x: len(x.src) <= MAX_LEN and len(x.tar) <= MAX_LEN

    # Caching to avoid redoing tokenization.
    prefix = 'de_en' if german_to_english else 'en_de'
    prefix += '_iwslt' if use_iwslt else '_wmt14'
    val_cache_path = os.path.join(dataset_path, f'{prefix}_val_cache.csv')
    test_cache_path = os.path.join(dataset_path, f'{prefix}_test_cache.csv')

    ts = time.time()
    if not use_caching_mechanism or not (os.path.exists(train_cache_path) and os.path.exists(val_cache_path)):
        # the dataset is made of .tar and .src attributes
        # so basically a table with 2 columns 'src' and 'tar'
        src_ext = '.de' if german_to_english else '.en'
        tar_ext = '.en' if german_to_english else '.de'
        dataset_split_fn = datasets.IWSLT.splits if use_iwslt else datasets.WMT14.splits
        train_dataset, val_dataset, test_dataset = dataset_split_fn(exts = (src_ext, tar_ext), fields = fields, root = dataset_path, filter_pred = filter_pred)

        save_cache(train_cache_path, train_dataset)
        save_cache(val_cache_path, val_dataset)
        save_cache(test_cache_path, test_dataset)
    else:
        train_dataset, val_dataset = DatasetWrapper.get_train_and_val_datasets(train_cache_path, val_cache_path, fields, filter_pred = filter_pred)
    print(f'Data preparation time : {time.time() - ts:3f}s.')

    MIN_FREQ = 2
    src_field_processor.build_vocab(train_dataset.src, min_freq = MIN_FREQ)
    tar_field_processor.build_vocab(train_dataset.tar, min_freq = MIN_FREQ)

    return train_dataset, val_dataset, src_field_processor, tar_field_processor
    




In [ ]:
def get_datalaoders(dataset_path, language_direction, dataset_name, batch_size, device):
    train_dataset, val_dataset, src_field_processor, tar_field_processor = get_datasets_and_vocabs(dataset_path, language_direction, dataset_name = DatasetType.IWSLT.name)
    train_token_ids_loader, val_token_ids_loader = BucketIterator.splits(datasets = (train_dataset, val_dataset), batch_size = batch_size, device = device, sort_within_batch = True, batch_size_fn = batch_size_fn)
    return train_token_ids_loader, val_token_ids_loader, src_field_processor, tar_field_processor

In [ ]:
global longest_src_sentence, longest_tar_sentence
def batch_size_fn(new_example, count, sofar):
    global longest_src_sentence, longest_tar_sentence
    if count == 1:
        longest_src_sentence = 0
        longest_tar_sentence = 0
    longest_src_sentence = max(longest_src_sentence, len(new_example.src))
    longest_tar_sentence = max(longest_tar_sentence, len(new_example.tar)+2)

    num_of_tokens_in_src_tensor = count*longest_src_sentence
    num_of_tokens_in_tar_tensor = count*longest_tar_sentence
    return max(num_tokens_in_src_tensor, num_tokens_in_tar_tensor)

## Caching mechanism

In [ ]:
class FastTranslationDataset(Dataset):
    @staticmethod
    def sort_key(ex):
        # interleaves 16bit representation of lengths.
        # heuristic that helps the BucketIterator sort bigger batches first.
        return interleave_keys(len(ex.src), len(ex.tar))

    def __init__(self, cache_path, fields, **kwargs):
        cached_data = [line.split() for line in open(cache_path, encoding = 'utf-8')]
        cached_data_src = cached_data[0::2] # even lines contain source examples.
        cached_data_tar = cached_data[1::2] # odd lines contain target examples
        assert len(cached_data_src) == len(cached_data_tar), 'Source and target data should be of the same length.'
        examples = []
        src_dataset_total_number_of_tokens = 0
        tar_dataset_total_number_of_tokens = 0
        for src_tokenized_data, tar_tokenized_data in zip(cached_data_src, cached_data_tar):
            ex = Example()
            
            setattr(ex, 'src', src_tokenized_data)
            setattr(ex, 'tar', tar_tokenized_data)

            examples.append(ex)

            # Update the number of tokens.
            src_dataset_total_number_of_tokens += len(src_tokenized_data)
            tar_dataset_total_number_of_tokens += len(tar_tokenized_data)

        # print relevant information about the dataset.
        filename_parts = os.path.split(cache_path)[1].split('_')
        src_language, tar_language = ('English', 'German') if filename_parts[0] == 'en' else ('German', 'English')
        dataset_name = 'IWSLT' if filename_parts[2] == 'iwslt' else 'WMT-14'
        dataset_type = 'train' if filename_parts[3] == 'train' else 'val'
        print(f'{dataset_type} dataset ({dataset_name}) has {src_dataset_total_number_of_tokens} tokens in the SOURCE language ({src_language}) corpus.')
        print(f'{dataset_type} dataset ({dataset_name}) has {tar_dataset_total_number_of_tokens} tokens in the TARGET language ({tar_language}) corpus.')

        # Call the parent class dataset's constructor.
        super().__init__(examples, fields, **kwargs)


In [ ]:
class DatasetWrapper(FastTranslationDataset):
    @classmethod
    def get_train_and_val_datasets(cls, train_cache_path, val_cache_path, fields, **kwargs):
        train_dataset = cls(train_cache_path, fields, **kwargs)
        val_dataset = cls(val_cache_path, fields, **kwargs)

        return train_dataset, val_dataset

def save_cache(cache_path, dataset):
    with open(cache_path, 'w', encoding='utf-8') as cache_file:
        for ex in dataset.examples:
            cache_file.write(' '.join(ex.src) + '\n')
            cache_file.write(' '.join(ex.tar) + '\n')